In [4]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/train.csv"

# Load CSV file into a pandas DataFrame
train_data = pd.read_csv(file_path)

train_data.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,timestamp_(min),feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24
0,0.0,0.732689,0.761748,0.606848,0.488746,0.424310,0.403609,0.519318,0.398792,0.451453,...,0.318797,0.451856,0.571500,0.469717,0.609883,0.008432,0.0,0.481838,0.006536,0.138249
1,1.0,0.732799,0.761855,0.607133,0.488781,0.432008,0.410256,0.511364,0.402568,0.455657,...,0.321463,0.456123,0.562226,0.466533,0.629812,0.008432,0.0,0.477218,0.006536,0.115207
2,2.0,0.732938,0.761594,0.606895,0.488791,0.418858,0.407724,0.488636,0.396526,0.456104,...,0.347219,0.456692,0.572002,0.487845,0.643598,0.006745,0.0,0.492623,0.008715,0.092166
3,3.0,0.732893,0.761656,0.606478,0.488802,0.417896,0.404242,0.500000,0.405589,0.460020,...,0.361904,0.460532,0.563354,0.479512,0.644690,0.008432,0.0,0.457064,0.008715,0.142857
4,4.0,0.732788,0.761573,0.606777,0.488800,0.421103,0.407407,0.511364,0.399547,0.458507,...,0.359767,0.458825,0.563354,0.448298,0.629948,0.006745,0.0,0.472223,0.006536,0.170507


In [5]:
import numpy as np

def generate_binary_noise_mask(no_samples, no_features, masked_proportion, mean_length):
    pm_to_m = 1 - 1 / mean_length
    pm_to_u = 1 / mean_length
    pu_to_m = (1 / mean_length) * (masked_proportion / (1 - masked_proportion))
    pu_to_u = 1 - (1 / mean_length) * (masked_proportion / (1 - masked_proportion))

    # Transition probability matrix
    P = np.array([[pm_to_m, pm_to_u],
                  [pu_to_m, pu_to_u]])

    # Generate binary noise mask matrix using Markov chain
    Markov = np.zeros((no_samples, no_features), dtype=int)
    state = np.random.choice([0, 1], size=no_samples)
    for i in range(1, no_samples):
        state[i] = np.random.choice([0, 1], p=P[state[i-1]])

    mask_lengths = np.random.geometric(p=1/mean_length, size=no_samples)


    for i, length in enumerate(mask_lengths):
        start_idx = np.random.randint(0, no_features - length + 1)
        end_idx = start_idx + length
        Markov[i, start_idx:end_idx] = 1

    return Markov


no_samples = len(train_data)
no_features = len(train_data.columns) - 1
masked_proportion = 0.2
mean_length = 2

binary_noise_mask = generate_binary_noise_mask(no_samples, no_features, masked_proportion, mean_length)
print(binary_noise_mask)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [1 1 1 ... 0 0 0]]


In [10]:
# Assuming `data_size` is the size of your dataset
data_size = 1000

# Generate random anomaly indices
num_anomalies = 50
anomalie= np.random.choice(data_size, size=num_anomalies, replace=False)
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers, Model

# # Data Augmentation with Geometric Distribution Masks
# def generate_binary_noise_mask(num_samples, num_features, masked_proportion, mean_length):
#     M = np.zeros((num_samples, num_features), dtype=int)
#     mask_lengths = np.random.geometric(p=1/mean_length, size=num_samples)
#     for i, length in enumerate(mask_lengths):
#         start_idx = np.random.randint(0, num_features - length + 1)
#         end_idx = start_idx + length
#         M[i, start_idx:end_idx] = 1
#     return M

# # Transformer-based Autoencoder
# class TransformerAutoencoder(Model):
#     def __init__(self, num_layers, d_model, num_heads, dff, input_shape, dropout_rate=0.1):
#         super(TransformerAutoencoder, self).__init__()
#         self.encoder = TransformerEncoder(num_layers, d_model, num_heads, dff, input_shape[-1], dropout_rate)
#         self.decoder = TransformerDecoder(num_layers, d_model, num_heads, dff, input_shape[-1], dropout_rate)

#     def call(self, inputs, training=False):
#         encoded = self.encoder(inputs, training=training)
#         decoded = self.decoder(encoded, training=training)
#         return decoded

# # Define the Transformer Encoder layer
# class TransformerEncoder(layers.Layer):
#     def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, dropout_rate=0.1):
#         super(TransformerEncoder, self).__init__()
#         self.enc_layers = [TransformerEncoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

#     def call(self, x, training=False):
#         for layer in self.enc_layers:
#             x = layer(x, training=training)
#         return x

# # Define the Transformer Encoder Layer
# class TransformerEncoderLayer(layers.Layer):
#     def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
#         super(TransformerEncoderLayer, self).__init__()
#         self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
#         self.ffn = tf.keras.Sequential([
#             layers.Dense(dff, activation='relu'),
#             layers.Dense(d_model)
#         ])
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(dropout_rate)
#         self.dropout2 = layers.Dropout(dropout_rate)

#     def call(self, x, training=False):
#         attn_output = self.mha(x, x, x)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(x + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         out2 = self.layernorm2(out1 + ffn_output)
#         return out2

# # Define the Transformer Decoder layer
# class TransformerDecoder(layers.Layer):
#     def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, dropout_rate=0.1):
#         super(TransformerDecoder, self).__init__()
#         self.dec_layers = [TransformerDecoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

#     def call(self, x, enc_output, training=False):
#         for layer in self.dec_layers:
#             x = layer(x, enc_output, training=training)
#         return x

# # Define the Transformer Decoder Layer
# class TransformerDecoderLayer(layers.Layer):
#     def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
#         super(TransformerDecoderLayer, self).__init__()
#         self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
#         self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
#         self.ffn = tf.keras.Sequential([
#             layers.Dense(dff, activation='relu'),
#             layers.Dense(d_model)
#         ])
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(dropout_rate)
#         self.dropout2 = layers.Dropout(dropout_rate)
#         self.dropout3 = layers.Dropout(dropout_rate)

#     def call(self, x, enc_output, training=False):
#         attn1, _ = self.mha1(x, x, x)
#         attn1 = self.dropout1(attn1, training=training)
#         out1 = self.layernorm1(x + attn1)
#         attn2, _ = self.mha2(out1, enc_output, enc_output)
#         attn2 = self.dropout2(attn2, training=training)
#         out2 = self.layernorm2(out1 + attn2)
#         ffn_output = self.ffn(out2)
#         ffn_output = self.dropout3(ffn_output, training=training)
#         out3 = self.layernorm3(out2 + ffn_output)
#         return out3

# # GAN Framework
# class GAN(Model):
#     def __init__(self, generator, discriminator):
#         super(GAN, self).__init__()
#         self.generator = generator
#         self.discriminator = discriminator

#     def compile(self, gen_optimizer, dis_optimizer, gen_loss_fn, dis_loss_fn):
#         super(GAN, self).compile()
#         self.gen_optimizer = gen_optimizer
#         self.dis_optimizer = dis_optimizer
#         self.gen_loss_fn = gen_loss_fn
#         self.dis_loss_fn = dis_loss_fn

#     def train_step(self, real_data):
#         # Sample noise for generator input
#         noise = tf.random.normal((tf.shape(real_data)[0], latent_dim))

#         # Generate fake data using generator
#         with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
#             fake_data = self.generator(noise, training=True)
#             real_output = self.discriminator(real_data, training=True)
#             fake_output = self.discriminator(fake_data, training=True)

#             # Compute generator loss
#             gen_loss = self.gen_loss_fn(fake_output)

#             # Compute discriminator loss
#             dis_loss = self.dis_loss_fn(real_output, fake_output)

#         # Update generator parameters
#         gen_gradients = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
#         self.gen_optimizer.apply_gradients(zip(gen_gradients, self.generator.trainable_variables))

#         # Update discriminator parameters
#         dis_gradients = dis_tape.gradient(dis_loss, self.discriminator.trainable_variables)
#         self.dis_optimizer.apply_gradients(zip(dis_gradients, self.discriminator.trainable_variables))

#         return {"gen_loss": gen_loss, "dis_loss": dis_loss}

# # Define Discriminator
# class Discriminator(Model):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         # Define layers for discriminator

#     def call(self, x):
#         # Forward pass through discriminator
#         return x

# # Define Generator
# class Generator(Model):
#     def __init__(self):
#         super(Generator, self).__init__()
#         # Define layers for generator

#     def call(self, x):
#         # Forward pass through generator
#         return x

# # Define custom loss functions
# def generator_loss(fake_output):
#     # Define generator loss
#     return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

# def discriminator_loss(real_output, fake_output):
#     # Define discriminator loss
#     real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
#     fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
#     return real_loss + fake_loss

# # Instantiate the models
# generator = Generator()
# discriminator = Discriminator()
# gan = GAN(generator, discriminator)

# # Compile the GAN model
# gan.compile(gen_optimizer=tf.keras.optimizers.Adam(),
#             dis_optimizer=tf.keras.optimizers.Adam(),
#             gen_loss_fn=generator_loss,
#             dis_loss_fn=discriminator_loss)

# # Train the GAN model
# latent_dim = 100
# gan.fit(train_data, epochs=10)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

file_path = "/content/drive/My Drive/train.csv"

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

def load_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)
    # Assuming the time series data doesn't need to handle timestamps for model input
    features = data.drop(['timestamp_(min)'], axis=1, errors='ignore')  # drop non-feature columns

    # Normalize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Convert to PyTorch tensors
    tensor_data = torch.tensor(scaled_features, dtype=torch.float32)
    return tensor_data

# Load and preprocess data
data_tensor = load_data(file_path)

# Create a DataLoader
train_loader = DataLoader(data_tensor, batch_size=64, shuffle=True)

def generate_binary_noise_mask(num_samples, num_features, masked_proportion, mean_length):
    # Calculate transition probabilities
    pm_to_u = 1 / mean_length  # Mask to unmask
    pm_to_m = 1 - pm_to_u      # Mask to mask
    pu_to_m = pm_to_u * (masked_proportion / (1 - masked_proportion))  # Unmask to mask
    pu_to_u = 1 - pu_to_m      # Unmask to unmask

    # Generate binary noise mask matrix using Markov chain for each feature
    mask = np.zeros((num_samples, num_features), dtype=int)

    for feature in range(num_features):
        # Start unmasked
        state = np.random.choice([0, 1], p=[1 - masked_proportion, masked_proportion])
        mask[0, feature] = state

        for i in range(1, num_samples):
            if state == 1:  # If currently masked
                state = np.random.choice([0, 1], p=[pm_to_u, pm_to_m])
            else:  # If currently unmasked
                state = np.random.choice([0, 1], p=[pu_to_m, pu_to_u])
            mask[i, feature] = state

    return torch.tensor(mask, dtype=torch.float32)

def masked_dataloader(data_tensor, batch_size, masked_proportion, mean_length):
    # Generate masks for all data
    num_samples, num_features = data_tensor.size()
    mask = generate_binary_noise_mask(num_samples, num_features, masked_proportion, mean_length)
    masked_data = data_tensor * mask  # Element-wise multiplication to apply the mask

    # Create a DataLoader
    dataset = TensorDataset(masked_data, data_tensor)  # Using original data as target
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


train_loader = masked_dataloader(data_tensor, batch_size=64, masked_proportion=0.2, mean_length=5)

class TransformerEncoder(nn.Module):
    def __init__(self, feature_size, num_heads, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.encoder_layers = nn.TransformerEncoderLayer(d_model=feature_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layers, num_layers=num_layers)

    def forward(self, src):
        return self.transformer_encoder(src)

class MLPDecoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLPDecoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)
        self.fc2 = nn.Linear(output_dim, output_dim)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        return self.activation(self.fc2(x))

class Generator(nn.Module):
    def __init__(self, feature_size, num_heads, num_encoder_layers, output_dim):
        super(Generator, self).__init__()
        self.encoder = TransformerEncoder(feature_size, num_heads, num_encoder_layers)
        self.decoder = MLPDecoder(feature_size, output_dim)

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        return self.activation(self.fc2(x))

# Reconstruction Loss
def reconstruction_loss(reconstructed, original):
    return nn.functional.mse_loss(reconstructed, original)

# Set up models, optimizers
generator = Generator(feature_size=25, num_heads=5, num_encoder_layers=3, output_dim=25)
discriminator = Discriminator(input_dim=25)

g_optimizer = optim.Adam(generator.parameters(), lr=0.001)
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

def train_epoch(generator, discriminator, loader, g_optimizer, d_optimizer):
    for masked_data, real_data in loader:
        # Forward pass through generator
        generated_data = generator(masked_data)

        # Discriminator training
        d_optimizer.zero_grad()
        real_pred = discriminator(real_data)
        fake_pred = discriminator(generated_data.detach())
        d_loss = -(torch.log(real_pred) + torch.log(1 - fake_pred)).mean()
        d_loss.backward()
        d_optimizer.step()

        # Generator training
        g_optimizer.zero_grad()
        fake_pred = discriminator(generated_data)
        g_loss = -torch.log(fake_pred).mean()
        rec_loss = nn.functional.mse_loss(generated_data, real_data)  # Reconstruction loss
        total_g_loss = g_loss + rec_loss
        total_g_loss.backward()
        g_optimizer.step()

# Execute training
num_epochs = 10
for epoch in range(num_epochs):
    train_epoch(generator, discriminator, train_loader, g_optimizer, d_optimizer)
    print(f'Epoch {epoch+1}/{num_epochs} completed.')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/10 completed.
Epoch 2/10 completed.
Epoch 3/10 completed.
Epoch 4/10 completed.
Epoch 5/10 completed.
Epoch 6/10 completed.
Epoch 7/10 completed.
Epoch 8/10 completed.
Epoch 9/10 completed.
Epoch 10/10 completed.


In [8]:
# # Generate synthetic data using the trained generator
synthetic_data = generator(train_data.values, binary_noise_mask)

# Perform anomaly detection on the synthetic data
reconstruction_error = np.mean(np.abs(train_data.values - synthetic_data), axis=1)
threshold = 0.5  # Set a threshold for classification
anomalies = reconstruction_error > threshold



In [11]:
# # Print the indices of anomalies
print("Indices of anomalies in synthetic data:", (anomalie))


Indices of anomalies in synthetic data: [371   7  83 795 204 304  53 120 562 502  25  24 946 136 169 469 596 917
 286 626 696 191  75 206 658 767 834 801 732 412 444 333 606 931 550 680
 787 511 560   8 264 232 442  55 189  97 138 377 521 854]
